In [54]:
import pandas as pd
import nltk
import string
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('french')
nlp = spacy.load('fr_core_news_sm')

sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']

In [78]:
dic_code_theme = {"Préparer mon séjour": 1,
                  "Réserver et payer": 2,
                  "Gérer ma réservation": 3,
                  "Mon séjour": 4,
                  "Assurances": 5}
dic_decode_theme = {val: key for key, val in dic_code_theme.items()}

In [79]:
def lemmatise_text(text):
    lst_lematised = [token.lemma_ for token in nlp(text)] 
    return ' '.join(lst_lematised).lower()


def stem_text(text):
    lst_stemmerised = [stemmer.stem(token) for token in word_tokenize(text)]    
    return ' '.join(lst_stemmerised)


def substitute_punctuation(text):
    return ' '.join(text.replace("'", ' ').translate(str.maketrans('', '', string.punctuation)).split())


def supp(text):
    return text.replace("«", "").replace("’", "").replace("•", "").replace("®", "")

In [80]:
## Import du vectorizer et du classifieur

from joblib import load
vectoriser_theme = load('vectorizer_classif_theme.joblib')
classifier_theme = load('model_classif_theme.joblib')
classifier_domaine = load('model_classif_domaine.joblib')

In [101]:
quest_user = "ça va ?"

# I. Classification de la question user en domaine (to be CenterPark or not)

In [102]:
quest_user_clean = supp(substitute_punctuation(stem_text(lemmatise_text(quest_user))))

In [103]:
X_quest_user = pd.Series(quest_user_clean)

In [104]:
X_quest_user_clean_vectorized_tfidf = vectoriser_theme.transform(X_quest_user)

In [105]:
domaine_quest_user = classifier_domaine.predict(X_quest_user_clean_vectorized_tfidf)
domaine_quest_user

array([0], dtype=int64)

##  A. Si dans le domaine

### 1.Classification de la question user en themes

Redimension car NN

In [28]:
XX_quest_user = X_quest_user_clean_vectorized_tfidf.toarray().reshape(X_quest_user_clean_vectorized_tfidf.shape[0],1,
                                                X_quest_user_clean_vectorized_tfidf.shape[1])

Prédiction avec le classifieur

In [29]:
pred_proba = classifier_theme.predict(XX_quest_user)

Transformation proba en classe

In [30]:
import numpy as np
idx = np.argmax(pred_proba, axis=-1)
YY_pred = np.zeros( pred_proba.shape )
YY_pred[ np.arange(YY_pred.shape[0]), idx] = 1

In [31]:
theme_quest_user = list(YY_pred[0]).index(1) +1
theme_quest_user

5

### 2. Trouver la question de la FAQ la plus proche/similaire de la question user

In [ ]:
faq_theme = faq.iloc[theme_quest_user-1][["question", 'reponse']]
quest_user_clean = supp_sw(supp(substitute_punctuation(stem_text(lemmatise_text(quest_user)))))
quest_user_clean_tokens = nlp(quest_user_clean)
lst_similarity = [quest_user_clean_tokens.similarity(token) for token in faq_theme.tokens]

### 3. Renvoyer la plus réponse de cette question FAQ (affichage)

In [ ]:
rep_quest_user = faq_theme.reponse[np.asarray(lst_similarity).argmax()]

## B. Pas dans le domaine : générer réponse originale avec le GAN sur text (partie Mathilde)